## Nobel API - check Wikidata and Nobel API
version. 0.2 changed api call and added **Vrandečić EWS**
* This [Jupyter Notebook](https://github.com/salgo60/open-data-examples/blob/master/Nobel%20API.ipynb)  

  * WIkidata property [Property:P8024](https://www.wikidata.org/wiki/Property:P8024)
  * [Developer xone nobelprize.org](https://www.nobelprize.org/about/developer-zone-2/)
      * maybe better API is [api.nobelprize.org/2.0/laureates](https://api.nobelprize.org/2.0/laureates)

In [1]:
from datetime import datetime
start_time = datetime.now()
print("Last run: ", datetime.now())

In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url ="https://api.nobelprize.org/2.0/laureates?offset=0&limit=1500"
dftot = pd.DataFrame()
r = http.request('GET', url)
data = json.loads(r.data)
dftot = dftot.append(pd.DataFrame(data["laureates"]),sort=False)

In [3]:
dftot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951 entries, 0 to 950
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               951 non-null    object
 1   knownName        927 non-null    object
 2   givenName        927 non-null    object
 3   familyName       925 non-null    object
 4   fullName         927 non-null    object
 5   gender           927 non-null    object
 6   birth            926 non-null    object
 7   links            951 non-null    object
 8   nobelPrizes      951 non-null    object
 9   death            628 non-null    object
 10  orgName          24 non-null     object
 11  nativeName       24 non-null     object
 12  acronym          10 non-null     object
 13  founded          23 non-null     object
 14  penName          11 non-null     object
 15  birthCountry     1 non-null      object
 16  birthCountryNow  1 non-null      object
 17  birthContinent   1 non-null      ob

In [4]:
dftot.tail(10)


,id,knownName,givenName,familyName,fullName,gender,birth,links,nobelPrizes,death,orgName,nativeName,acronym,founded,penName,birthCountry,birthCountryNow,birthContinent
941,50,"{'en': 'Wolfgang Pauli', 'se': 'Wolfgang Pauli'}","{'en': 'Wolfgang', 'se': 'Wolfgang'}","{'en': 'Pauli', 'se': 'Pauli'}","{'en': 'Wolfgang Pauli', 'se': 'Wolfgang Pauli'}",male,"{'date': '1900-04-25', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1945', 'category': {'en': 'Phy...","{'date': '1958-12-15', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
942,483,"{'en': 'Woodrow Wilson', 'se': 'Woodrow Wilson'}","{'en': 'Woodrow', 'se': 'Woodrow'}","{'en': 'Wilson', 'se': 'Wilson'}","{'en': 'Thomas Woodrow Wilson', 'se': 'Thomas ...",male,"{'date': '1856-12-28', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1919', 'category': {'en': 'Pea...","{'date': '1924-02-03', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
943,557,"{'en': 'Yasser Arafat', 'se': 'Yasser Arafat'}","{'en': 'Yasser', 'se': 'Yasser'}","{'en': 'Arafat', 'se': 'Arafat'}","{'en': 'Yasser Arafat', 'se': 'Yasser Arafat'}",male,"{'date': '1929-08-24', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1994', 'category': {'en': 'Pea...","{'date': '2004-11-11', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
944,642,"{'en': 'Yasunari Kawabata', 'se': 'Yasunari Ka...","{'en': 'Yasunari', 'se': 'Yasunari'}","{'en': 'Kawabata', 'se': 'Kawabata'}","{'en': 'Yasunari Kawabata', 'se': 'Yasunari Ka...",male,"{'date': '1899-06-11', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1968', 'category': {'en': 'Lit...","{'date': '1972-04-16', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
945,559,"{'en': 'Yitzhak Rabin', 'se': 'Yitzhak Rabin'}","{'en': 'Yitzhak', 'se': 'Yitzhak'}","{'en': 'Rabin', 'se': 'Rabin'}","{'en': 'Yitzhak Rabin', 'se': 'Yitzhak Rabin'}",male,"{'date': '1922-03-01', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1994', 'category': {'en': 'Pea...","{'date': '1995-11-04', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
946,826,"{'en': 'Yoichiro Nambu', 'se': 'Yoichiro Nambu'}","{'en': 'Yoichiro', 'se': 'Yoichiro'}","{'en': 'Nambu', 'se': 'Nambu'}","{'en': 'Yoichiro Nambu', 'se': 'Yoichiro Nambu'}",male,"{'date': '1921-01-18', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2008', 'category': {'en': 'Phy...","{'date': '2015-07-05', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
947,927,"{'en': 'Yoshinori Ohsumi', 'se': 'Yoshinori Oh...","{'en': 'Yoshinori', 'se': 'Yoshinori'}","{'en': 'Ohsumi', 'se': 'Ohsumi'}","{'en': 'Yoshinori Ohsumi', 'se': 'Yoshinori Oh...",male,"{'date': '1945-02-09', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2016', 'category': {'en': 'Phy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
948,265,"{'en': 'Yuan T. Lee', 'se': 'Yuan T. Lee'}","{'en': 'Yuan T.', 'se': 'Yuan T.'}","{'en': 'Lee', 'se': 'Lee'}","{'en': 'Yuan T. Lee', 'se': 'Yuan T. Lee'}",male,"{'date': '1936-11-19', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1986', 'category': {'en': 'Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
949,794,"{'en': 'Yves Chauvin', 'se': 'Yves Chauvin'}","{'en': 'Yves', 'se': 'Yves'}","{'en': 'Chauvin', 'se': 'Chauvin'}","{'en': 'Yves Chauvin', 'se': 'Yves Chauvin'}",male,"{'date': '1930-10-10', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2005', 'category': {'en': 'Che...","{'date': '2015-01-27', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950,726,"{'en': 'Zhores Alferov', 'se': 'Zhores Alferov'}","{'en': 'Zhores', 'se': 'Zhores'}","{'en': 'Alferov', 

In [5]:
#dftot.sample(10)

In [6]:
listNobel = []
for index,row in dftot.iterrows():
    new_item = dict()
    new_item['id'] = row['id']
    try:
        new_item['name_en'] = row['knownName']['en']
    except:
        pass
    try:
        new_item['name_se'] = row['knownName']['se']
    except:
        pass
    new_item['link'] = row['links']['href']
    try:
        new_item['gender'] = row['gender']
    except:
        pass
    listNobel.append(new_item)
NobelTot = pd.DataFrame(listNobel,
                  columns=['id','name_en','name_se','link','gender'])

NobelTot.info()    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951 entries, 0 to 950
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       951 non-null    object
 1   name_en  927 non-null    object
 2   name_se  927 non-null    object
 3   link     951 non-null    object
 4   gender   927 non-null    object
dtypes: object(5)
memory usage: 37.3+ KB


## EWS Vrandečić  
* [tweet](https://twitter.com/vrandezo/status/1313587353433251840?s=20)

In [7]:
NobelTot[NobelTot['name_en'].str.contains("Vrandečić", na=False)] 

,id,name_en,name_se,link,gender


In [8]:
#check number of prizes received
NobelTot[NobelTot['name_en'].str.contains("Vrandečić", na=False)]["name_en"].sum()


0

## Check Wikidata matches

In [9]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?item ?itemLabel ?NobelAPI WHERE {
  ?item wdt:P8024 ?NobelAPI.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} order by xsd:integer(?NobelAPI)"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']
    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDNobel = get_sparql_dataframe(endpoint_url, query)

In [10]:
WDNobel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item       952 non-null    object
 1   itemLabel  952 non-null    object
 2   NobelAPI   952 non-null    object
dtypes: object(3)
memory usage: 22.4+ KB


### Compare WD <-> Nobel 

In [11]:
dfmerge = pd.merge(WDNobel, NobelTot,how='outer', left_on="NobelAPI",right_on="id",indicator=True)
dfmerge['_merge'] = dfmerge['_merge'].str.replace('left_only','WD_only').str.replace('right_only','Nobel_only')
dfmerge.rename(columns={"_merge": "WD_Nobel"},inplace = True)    
dfmerge[-10:]  

,item,itemLabel,NobelAPI,id,name_en,name_se,link,gender,WD_Nobel
942,http://www.wikidata.org/entity/Q1635006,Michael Kremer,984,984,Michael Kremer,Michael Kremer,http://masterdataapi.nobelprize.org/2/laureate...,male,both
943,http://www.wikidata.org/entity/Q5677307,Harvey J. Alter,985,985,Harvey J. Alter,Harvey J. Alter,http://masterdataapi.nobelprize.org/2/laureate...,male,both
944,http://www.wikidata.org/entity/Q1927895,Michael Houghton,986,986,Michael Houghton,Michael Houghton,http://masterdataapi.nobelprize.org/2/laureate...,male,both
945,http://www.wikidata.org/entity/Q22669757,Charles M. Rice,987,987,Charles M. Rice,Charles M. Rice,http://masterdataapi.nobelprize.org/2/laureate...,male,both
946,http://www.wikidata.org/entity/Q193803,Roger Penrose,988,988,Roger Penrose,Roger Penrose,http://masterdataapi.nobelprize.org/2/laureate...,male,both
947,http://www.wikidata.org/entity/Q65807,Reinhard Genzel,989,989,Reinhard Genzel,Reinhard Genzel,http://masterdataapi.nobelprize.org/2/laureate...,male,both
948,http://www.wikidata.org/entity/Q493956,Andrea M. Ghez,990,990,Andrea Ghez,Andrea Ghez,http://masterdataapi.nobelprize.org/2/laureate...,female,both
949,http://www.wikidata.org/entity/Q17280087,Emmanuelle Charpentier,991,991,Emmanuelle Charpentier,Emmanuelle Charpentier,http://masterdataapi.nobelprize.org/2/laureate...,female,both
950,http://www.wikidata.org/entity/Q56068,Jennifer Doudna,992,992,Jennifer A. Doudna,Jennifer A. Doudna,http://masterdataapi.nobelprize.org/2/laureate...,female,both
951,http://www.wikidata.org/entity/Q2344210,Louise Glück,993,NaN,NaN,NaN,NaN,NaN,WD_only


In [12]:
dfmerge["WD_Nobel"].value_counts()  

both       951
WD_only      1
Name: WD_Nobel, dtype: int64

In [13]:
Nobelonly = dfmerge[dfmerge["WD_Nobel"]=="Nobel_only"].copy() 

from IPython.display import HTML
Nobelonly["Nobel"] = "<a href='https://api.nobelprize.org/v1/laureate.json?id=" + Nobelonly["id"] + "'>link</a>"
Nobelonly["Nobelhtml"] = "<a href='https://www.nobelprize.org/laureate/" + Nobelonly["id"] + "'>html</a>"
pd.set_option("display.max.columns", None) 
HTML(Nobelonly.to_html(escape=False))

,item,itemLabel,NobelAPI,id,name_en,name_se,link,gender,WD_Nobel,Nobel,Nobelhtml


In [14]:
print("End run: ", datetime.now())

End run:  2020-10-08 13:05:40.215406


In [15]:
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Time elapsed (hh:mm:ss.ms) 0:00:04.507851
